In [ ]:
# requirements.txt
# !cp ./drive/MyDrive/Academic/cplex_studio2210.linux_x86_64.bin ./
# !chmod u+x ./cplex_studio2210.linux_x86_64.bin
# !./cplex_studio2210.linux_x86_64.bin
# install CPLEX via setup.py

Drive path to ```/ibm```: https://drive.google.com/drive/folders/1zUIlliuQhfSQK45JnF4B9OuQgBShN55-?usp=sharing (Add this folder to drive shortcut)

In [1]:
!python './drive/MyDrive/ibm/ILOG/CPLEX_Studio221/python/setup.py' install
# !python './drive/MyDrive/Academic/ibm/ILOG/CPLEX_Studio221/python/setup.py' install

['/content/drive/MyDrive/ibm/ILOG/CPLEX_Studio221/cplex/python/3.7/x86-64_linux/cplex']
['/content/drive/MyDrive/ibm/ILOG/CPLEX_Studio221/cplex/python/3.7/x86-64_linux/setup.py']
Invoking ['/usr/bin/python3', 'setup.py', 'install'] in /content/drive/MyDrive/ibm/ILOG/CPLEX_Studio221/cplex/python/3.7/x86-64_linux
/usr/lib/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'zip_safe'
  warnings.warn(msg)
running install
running build
running build_py
running install_lib
creating /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cplex/model_info.py -> /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cplex/paramset.py -> /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cplex/aborter.py -> /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cplex/constant_class.py -> /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cplex/callbacks.py -> /usr/local/lib/python3.7/dist-packages/cplex
copying build/lib/cple

### Data Generation Guideline
------------------------------<br>
N kind of items <br>
M+1 points <br>
q[i] number of item i to take <br>
Q[i,j] number of item i at point j <br>
d[i, j] distance from point i to j<br>
------------------------------<br>
Decision variables: <br>
x[j], go or not go to point j <br>
y[i, j], go or not from i to j <br>
------------------------------<br>
Constraint: <br>
x[0]=1<br>
y[i,i]=0, (i=0->m)<br>
sum(y[i,j])=x[i], (i=0->m)<br>
sum(y[i,j])=sum(y[j,i]), (i=0->m, j from 0 to m)<br>
sum(Q[i,j]\*x[j])>=q[i], for every item i in request (j=1->m)<br>(Subtour Elimination) u[i] - u[j] + m\*y[i][j] <= m-1 <br>------------------------------<br>
Optimize Objective: Minimize sum(y[i,j]*d[j,j])<br>


In [ ]:
# total_data = get_json('./drive/MyDrive/Academic/IT4663/6_28_impossible_data.json')
# # assert len(total_data)>17
# json2txt(total_data,'./drive/MyDrive/Academic/IT4663/6_28_impossible_data')

### Utils

In [2]:
import cplex
from cplex.exceptions import CplexError
import sys, json
import itertools

def findsubsets(s, n):
    return list(itertools.combinations(s, n))

def checkd(d):
    subs = findsubsets(range(len(d)),3)
    for a,b,c in subs:
        if d[a][b]>d[b][c]+d[c][a]:
            return False
    return True
    
def dump_json(test_data,output_json_name=None):
    if output_json_name is not None:
            with open(output_json_name, "w",encoding='utf8') as output_file:
                json.dump(test_data,output_file,ensure_ascii=False)

def get_json(path):
    with open(path,"r", encoding='utf8') as f:
        data=json.load(f)
    return data

def read_file(path):
    with open(path,'r') as f:
        d = f.read().strip()
    return d

def write_file(content,path):
    with open(path,'w') as f:
        f.write(content)

In [3]:
def read_raw_data_and_solve(str_path: str, output_txt_path: str = None):
    """
    str_path: Path to txt file containing raw data
    output_txt_path: Path to save txt file containing data with solution

    Return: Dict{
        ... // same as init_data_with_solution
    }
    """
    d = read_file(str_path).split('\n')
    for i, row in enumerate(d):
        if i==0:
            n,m = int(row.split()[0]),int(row.split()[1])
            itloc = [[0 for i in range(m+1)] for j in range(n+1)]
            d = [[0 for i in range(m+1)] for j in range(m+1)]
        elif i>=1 and i<=n:
            itloc[i] = [0]+[int(x) for x in row.split()]
        elif i>=n+1 and i<=n+1+m:
            d[i-n-1] = [int(x) for x in row.split()]
        elif i==n+m+2:
            item2take = [0]+[int(x) for x in row.split()]
        else:
            print("Warning! There are more than m+n+3 rows in input data")
            break

    itemavail = [sum(itloc[i][1:]) for i in range(n+1)]
    data = {'n':n,'m':m,'item2take':item2take,'d':d,'itloc':itloc,'itemavail':itemavail}
    solver = Solver(data)
    data.update({"optimal_path":solver.get_optimal_path(),
                "optimal_value":solver.get_optimal_value(),
                "explanation":solver.get_explanation(),
                    "CPLEX_solving_time":solver.get_solving_time()})
    
    if output_txt_path is not None:
        tmp = []
        tmp.append("{} {}".format(data['n'],data['m']))
        for i in range(1,data['n']+1):
            tmp.append(' '.join([str(x) for x in data['itloc'][i][1:]]))
        for i in range(data['m']+1):
            tmp.append(' '.join([str(x) for x in data['d'][i]]))
        assert len(data['item2take'])==data['n']+1
        tmp.append(' '.join([str(x) for x in data['item2take'][1:]]))
        tmp.append(' '.join([str(x) for x in data['optimal_path']]))
        tmp.append(str(data['optimal_value']))
        tmp.append(str(data['CPLEX_solving_time']))
        write_file('\n'.join(tmp),output_txt_path)
    
    return data

In [4]:
from typing import Dict, List, Union
import random
from tqdm.auto import tqdm
import os
from os.path import join, exists
import math
import numpy as np

def json2txt(json_path: Union[str,List], txt_folder: str):
    if isinstance(json_path,str):
        json_data = get_json(json_path)
    else:
        assert isinstance(json_path,list), "List of data points or path please!"
        json_data = json_path
    
    if not exists(txt_folder):
        os.makedirs(txt_folder)
    for idx,data in enumerate(json_data):
        tmp = []
        tmp.append("{} {}".format(data['n'],data['m']))
        for i in range(1,data['n']+1):
            tmp.append(' '.join([str(x) for x in data['itloc'][i][1:]]))
        for i in range(data['m']+1):
            tmp.append(' '.join([str(x) for x in data['d'][i]]))
        assert len(data['item2take'])==data['n']+1
        tmp.append(' '.join([str(x) for x in data['item2take'][1:]]))
        # The last 3 lines are optimal path, optimal value and solving time (in seconds)
        if 'optimal_path' in data:
            tmp.append(' '.join([str(x) for x in data['optimal_path']]))
            tmp.append(str(data['optimal_value']))
            tmp.append(str(data['CPLEX_solving_time']))
        write_file('\n'.join(tmp),join(txt_folder,f'case_{idx}.txt'))

def get_rand_point(bound):
    tmp = [random.randint(0,bound),random.randint(0,bound)]
    while tmp==[0,0]:
        tmp = [random.randint(0,bound),random.randint(0,bound)]
    return tmp

def calc_eucli(a,b):
    # return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)
    return np.linalg.norm(np.array(a)-np.array(b))

In [5]:
def init_raw_data(n: int, m: int, low: int, high: int, bound: int, symmetric: bool = True,
                  take_percent: float=1.0, loop:bool=True, **kwargs) -> Dict:
    """
    Init raw data without solution
    ------
    n: number of types of items
    m: number of locations containing items (shelves, but excluding starting point)
    low: minimum number of item of each kind contained in a shelf
    high: maximum number of item of each kind contained in a shelf
    """
    itloc = [[random.randint(low,high) for i in range(m+1)] for j in range(n+1)] # (n+1)*(m+1) item-location matrix
    xys = [[0,0]]+[get_rand_point(bound) for _ in range(m)]
    assert len(xys)==m+1
    if not loop:
        d = [[calc_eucli(xys[i],xys[j]) for i in range(m+1)] for j in range(m+1)]
    else:
        d = [[random.randint(1,bound) for i in range(m+1)] for j in range(m+1)] # (m+1)*(m+1) distance matrix
    for i in range(m+1): # distance to same loc = 0
        d[i][i]=0
    for i in range(m+1): # placeholder line
        itloc[0][i]=0
    for i in range(1,n+1): # base contains no item
        itloc[i][0]=0
    if symmetric:
        for i in range(m):
            for j in range(i+1, m+1):
                d[i][j] = d[j][i]
    itemavail = [sum(itloc[i][1:]) for i in range(n+1)]
    item2take = [random.randint(0,int(take_percent*itemavail[i])) for i in range(n+1)] # generate random requests
    return {'n':n,'m':m,'low':low,'high':high,
        'itloc':itloc, 'd':d, 'itemavail':itemavail,'item2take':item2take}

def init_data_with_solution(n: int, m: int, low: int, high: int, bound: int , 
                            percent_usage: float=0.5, return_solver: bool=False, 
                            symmetric: bool=True, take_percent: float=1.0, loop:bool=True, **kwargs) -> Dict:
    """
    Init data with solution
    ------
    n: number of types of items
    m: number of locations containing items (shelves, but excluding starting point)
    low: minimum number of item of each kind contained in a shelf
    high: maximum number of item of each kind contained in a shelf
    take_percent: max percentage of each item in request
    percent_usage: if < 1.0, generate data such that optimal solution will include 
                   visits of no more than the same percentage of locations
                   e.g. percent_usage = 0.5 -> optimal_path will only include up to 50% locations

    Return: Dict{
        'n':n,
        'm':m,
        'd': distance matrix,
        'item2take': (n+1) matrix, where index [i] indicates
                     quantity of item i to take in request,
        'itloc': (n+1)*(m+1) matrix, where index [i][j] indicates
                number of item i at point j (i=1->n , j=1->m)
        'itemavail': (n+1) matrix, where index [i] indicates total number of item i available
        'optimal_path': optimal path found by CPLEX,
        'optimal_value': optimal value found by CPLEX (minimum total path distance),
        'explanation': explanation for solution,
        'CPLEX_solving_time': CPLEX solving time (seconds)
    }
    """

    tq = tqdm(desc='Trying ...')
    while True:
        data = init_raw_data(n,m,low,high,bound, symmetric, take_percent,loop)
        solver = Solver(data)
        data.update({"optimal_path":solver.get_optimal_path(),
                    "optimal_value":solver.get_optimal_value(),
                    "explanation":solver.get_explanation(),
                     "CPLEX_solving_time":solver.get_solving_time()})
        data.pop('low',None)
        data.pop('high',None)
        data.pop('bound',None)
        tq.update(1)
        if percent_usage<1.0 and len(data['optimal_path'])-2>int(data['m']*percent_usage):
            continue
        break
    tq.close()
    if return_solver:
        return [data, solver]
    return data

### CPLEX Solver

In [6]:
from typing import Dict, List
import cplex
import time, math

class Solver:
    """
    Solver that handles data with UNLIMITED decision variables (well, solving time gets escalated quickly)
    """
    def __init__(self, data: Dict):
        for k,v in data.items():
            setattr(self, k, v)
        self.data = data
        self.init_problem()
        self.init_constraint()
        self.solve()
        self.recheck()
        # print(f"Case with {self.n} kinds of item and {self.m} container locations")
        # print("Number of decision variables",len(self.prob.variables.get_lower_bounds()))

    def init_problem(self):
        self.prob = cplex.Cplex()
        m=self.m
        # x[j]
        self.prob.variables.add(names=[f"x_{j}" for j in range(1,m+1)],
                            types=[self.prob.variables.type.binary for j in range(1,m+1)])
        # y[i,j]
        self.prob.variables.add(names=[f"y_{i}_{j}" \
                                    for i in range(m+1) for j in range(m+1) if i!=j],
                            types=[self.prob.variables.type.binary \
                                for i in range(m+1) for j in range(m+1) if i!=j])
        # u[i]
        self.prob.variables.add(names=[f"u_{i}" for i in range(1,m+1)],
                                types=[self.prob.variables.type.continuous for i in range(1,m+1)])
        for u_var in [f"u_{i}" for i in range(1,m+1)]:
            self.prob.variables.set_lower_bounds(u_var, 1.0)
            self.prob.variables.set_upper_bounds(u_var, m)

        self.prob.objective.set_linear([(f"y_{i}_{j}",self.d[i][j])\
                             for i in range(m+1)\
                             for j in range(m+1) if i!=j])
        self.prob.objective.set_sense(self.prob.objective.sense.minimize)

    def init_constraint(self):
        # sum(y[i,j])=x[i], (i=0->m)
        m=self.m
        n=self.n
        for i in range(0,m+1):
            self.prob.linear_constraints.add(
                lin_expr= [cplex.SparsePair(ind= [f"y_{i}_{j}" for j in range(0,m+1) if j!=i]+[f"x_{i}"]\
                                            if i!=0 else [f"y_{i}_{j}" for j in range(0,m+1) if j!=i], \
                                            val= [1 for j in range(0,m+1) if j!=i]+[-1]\
                                            if i!=0 else [1 for j in range(0,m+1) if j!=i])],
                rhs= [0] if i!=0 else [1],
                names = ["Moving constraint (1)-{}".format(i)],
                senses = ['E']
            )
        # sum(y[i,j])=sum(y[j,i]), (i=0->m)
        for i in range(0,m+1):
            self.prob.linear_constraints.add(
                lin_expr= [cplex.SparsePair(ind= [f"y_{i}_{j}" for j in range(0,m+1) if j!=i]+\
                                                [f"y_{j}_{i}" for j in range(0,m+1) if j!=i], \
                                            val= [1 for j in range(0,m+1) if j!=i]+\
                                                [-1 for j in range(0,m+1) if j!=i])],
                rhs= [0],
                names = ["Moving constraint (2)-{}".format(i)],
                senses = ['E']
            )
        # subtour elimination
        # i.e. u[i] + m*y[i][j] - u[j] <= m - 1
        for i in range(1,m+1):
            for j in range(1,m+1):
                if i==j:
                    continue
                self.prob.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[f"u_{i}",f"y_{i}_{j}",f"u_{j}"],
                                               val=[1,m,-1])],
                    rhs=[m-1],
                    names = ["Subtour elimination -{}".format(i)],
                    senses = ['L']
                )
        # sum(Q[i,j]*x[j])>=q[i]
        for i in range(1,n+1):
            self.prob.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[f"x_{j}" for j in range(1,m+1)],
                                        val=[self.itloc[i][j] for j in range(1,m+1)])],
                rhs = [self.item2take[i]],
                names = ["Item taken satisfied-{}".format(i)],
                senses = ['G']
            )

    def solve(self):
        self.prob.set_log_stream(None)
        self.prob.set_results_stream(None)
        st = time.time()
        self.prob.solve()
        self.solve_time = time.time()-st
        self.x_opt = [self.roundt(self.prob.solution.get_values(f"x_{i}")) if i!=0 else 1 for i in range(self.m+1)]
        self.y_opt = [[self.roundt(self.prob.solution.get_values(f"y_{i}_{j}")) if i!=j else 0 for j in range(self.m+1)] for i in range(self.m+1)]
        self.optimal_path = self.get_path(self.y_opt,self.m)
        self.optimal_value = sum([self.d[self.optimal_path[i]][self.optimal_path[i+1]] for i in range(len(self.optimal_path)-1)])
        self.cplex_optimal_value = self.prob.solution.get_objective_value()

    def get_optimal_value(self) -> int:
        return self.optimal_value

    def get_optimal_path(self) -> List[int]:
        return self.optimal_path    

    def get_solving_time(self) -> float:
        """
        Return solving time in seconds
        """
        return self.solve_time

    def get_path(self, y_opt: List[List[int]]=None, m: int=None):
        m, y_opt = self.m, self.y_opt
        path = [[i,j] for i in range(m+1) for j in range(m+1) if y_opt[i][j]!=0]
        # print(path)
        start = 0
        dest = {}
        dest.update(path)
        prev = start
        pathtoget = [prev]
        while dest[prev]!=start:
            pathtoget.append(dest[prev])
            prev=dest[prev]
        return pathtoget + [start]

    def recheck(self):
        assert abs(self.cplex_optimal_value-self.optimal_value)<1e-6,\
               [self.cplex_optimal_value,self.optimal_value]
        for i in range(self.m+1):
            for j in range(self.m+1):
                if self.y_opt[i][j]!=0:
                    assert i in self.optimal_path, "Subtour detected!"
                    assert j in self.optimal_path, "Subtour detected!"

    def roundt(self, x):
        assert (x<1e-6) or (x>0.9999)
        return 0 if x<1e-6 else 1

    def explain(self):
        print(self.get_explanation())
    
    def get_explanation(self,data: Dict = None, x_opt: List[int] = None, y_opt: List[List[int]] = None):
        if data is None:
            data, x_opt, y_opt = self.data, self.x_opt, self.y_opt
        n, m, itloc = data['n'], data['m'], data['itloc']
        itemavail, d, item2take = data['itemavail'], data['d'], data['item2take']
        path = self.get_path(y_opt,m)
        distance_cost =[d[path[i]][path[i+1]] for i in range(len(path)-1)]
        INFO = []
        INFO.append("EXPLANATION")
        INFO.append("="*30)
        INFO.append("Path of the packager: {}".format(" -> ".join(map(str,path))))
        INFO.append("Distance of path: {}".format(" + ".join(map(str,distance_cost)))\
                    + " = {}".format(sum(distance_cost)))
        INFO.append("-"*30)
        INFO.append("The packager need to take: ")
        for i in range(1,n+1):
            INFO.append("\tItem {} - Quantity: {}".format(i,item2take[i]))
            INFO.append("\tHe visits places containing the item: {}".format\
                        (" - ".join(["{} ({})".format(j,itloc[i][j]) for j in path[1:-1] if itloc[i][j]!=0])))
            INFO.append("\tHe could take a quantity of up to {}, which satisfies request".\
                        format(sum([itloc[i][j] for j in path[1:-1]])))
        INFO.append("-"*30)
        INFO.append("Thus, the optimal value is {}".format(sum(distance_cost)))
        return "\n".join(INFO)

In [7]:
# # prProb.set_results_stream(None)
# prProb.solve()
# print("Objective value:",prProb.solution.get_objective_value())
# print("\n"+"="*20+" decision variables "+"="*20)
# # optimal variables
# import math
# x_opt = [int(prProb.solution.get_values(f"x_{i}")) if i!=0 else 1 for i in range(m+1) ]
# y_opt = [[int(prProb.solution.get_values(f"y_{i}_{j}")) if i!=j else 0 for j in range(m+1)] for i in range(m+1)]
# for i in range(m+1):
    # if i!=0:
        # print(f"x_{i}",prProb.solution.get_values(f"x_{i}"))
    # for j in range(m+1):
    #     if j!=i:
    #         val = prProb.solution.get_values(f"y_{i}_{j}")
    #         assert math.ceil(val)==y_opt[i][j], ((i,j),val,y_opt[i][j])
    #         if val!=0:
    #             print(f"y_{i}_{j}",val)

### Playground (test)

In [ ]:
data_args = {'n':20,
             'm':20,
             'low':0,
             'high':10,
             'bound':100,
             'take_percent':0.5,
             'percent_usage':1.0, # max % shelves allowed
             'loop':False,
             'symmetric':True,
             'return_solver':False}

data = init_data_with_solution(**data_args)
print(data.keys())
print('Number of kinds of items:',data['n'])
print('Number of locations excluding base:',data['m'])
print('Request:',data['item2take'][1:])
print('Quantity available per item:',data['itemavail'][1:])
print("Distance matrix:",data['d'])
print("Optimal value:",data['optimal_value'])
print("Optimal path:",data['optimal_path'])
print("Number of shelves visited (excluding base):",len(data['optimal_path'])-2)
print()
# print(data['explanation'])
print("Solving time: {}(s)".format(data['CPLEX_solving_time']))
# data = init_raw_data(**data_args)

Trying ...: 0it [00:00, ?it/s]

dict_keys(['n', 'm', 'itloc', 'd', 'itemavail', 'item2take', 'optimal_path', 'optimal_value', 'explanation', 'CPLEX_solving_time'])
Number of kinds of items: 20
Number of locations excluding base: 20
Request: [6, 13, 5, 40, 3, 49, 23, 43, 10, 17, 32, 37, 23, 9, 1, 31, 18, 17, 21, 20]
Quantity available per item: [106, 75, 101, 110, 78, 100, 116, 101, 99, 104, 93, 115, 101, 117, 89, 117, 94, 90, 94, 83]
Distance matrix: [[0, 133.00375934536586, 74.94664769020693, 70.9365914038728, 103.58571330062848, 110.27692415006868, 82.20097323024831, 49.09175083453431, 88.48163651289458, 86.76404785393545, 123.00406497347964, 61.91122676865643, 26.40075756488817, 83.24061508662703, 104.4030650891055, 96.17692030835673, 56.92099788303083, 95.0, 92.13576938409967, 59.61543424315552, 51.62363799656123], [133.00375934536586, 0, 75.69015788066504, 75.27283706623525, 44.04543109109048, 41.19465984809196, 78.64477096412705, 102.95630140987001, 50.32891812864648, 56.302753041036986, 25.298221281347036, 73.

In [ ]:
# imp_combinations = [
#                 [100,200],
#                 [200,250],
#                 [250,300],
#                 [300,400]
#                 [400,450],
#                 [500,500],
#                 [600,600],
#                 [800,800],
#                 [800,1000]]
# combinations = [[10,5],
#                 [10,15],
#                 [15,20],
#                 [20,25],
                # [25,30],
                # [35,40],
                
combinations = [
                [35,60],
                [45,60],
                [50,100],
                [80,100],
                [80,150],
                [100,150]]

solvers = []
total_data = []
for n,m in combinations:
    percent_usage=1.0
    if m<150:
        take_percent=0.3
    else:
        take_percent=0.1
    data_args = {'n':n,
                'm':m,
                'low':0,
                'high':10,
                'bound':100,
                'take_percent':take_percent,
                'percent_usage':percent_usage,
                'symmetric':True,
                'return_solver':False}
    # data = init_raw_data(**data_args)
    data = init_data_with_solution(**data_args)
    # print(data['CPLEX_solving_time'])
    total_data.append(data)
    print('Number of locations excluding base:',data['m'])
    print("Optimal value:",data['optimal_value'])
    print("Optimal path:",data['optimal_path'])
    print("Number of shelves visited (excluding base):",len(data['optimal_path'])-2)
    # solvers.append(solver)
    # dump_json(total_data,'./drive/MyDrive/Academic/IT4663/6_28_impossible_data.json')
    dump_json(total_data,'./drive/MyDrive/Academic/IT4663/7_18_2_data.json')
    # print("Optimal value:",data['optimal_value'],solver.prob.solution.get_objective_value())

# print()
# solver.explain()

Trying ...: 0it [00:00, ?it/s]

Number of locations excluding base: 30
Optimal value: 264.16147250380254
Optimal path: [0, 14, 4, 17, 19, 12, 16, 5, 18, 6, 25, 8, 27, 30, 3, 0]
Number of shelves visited (excluding base): 14


Trying ...: 0it [00:00, ?it/s]

Number of locations excluding base: 40
Optimal value: 232.95809547587888
Optimal path: [0, 18, 28, 24, 8, 6, 40, 31, 26, 12, 33, 7, 35, 36, 34, 21, 13, 22, 10, 16, 0]
Number of shelves visited (excluding base): 19


Trying ...: 0it [00:00, ?it/s]

In [ ]:
imp_combinations = [
                [200,200],
                [200,250],
                [250,300],
                [300,400]
                [400,450],
                [500,500],
                [600,600],
                [800,800],
                [800,1000]]
                
solvers = []
total_data = []
for n,m in imp_combinations:
    percent_usage=1.0
    if m<30:
        take_percent=0.8
    elif m<80:
        take_percent=0.5
    elif m<150:
        take_percent=0.3
    else:
        take_percent=0.1
    data_args = {'n':n,
                'm':m,
                'low':0,
                'high':10,
                'bound':100,
                'take_percent':take_percent,
                'percent_usage':percent_usage,
                'symmetric':True,
                'return_solver':False}
    data = init_raw_data(**data_args)
    # solvers.append(solver)
    dump_json(total_data,'./drive/MyDrive/Academic/IT4663/7_18_impossible_data.json')
    # dump_json(total_data,'./drive/MyDrive/Academic/IT4663/7_18_data.json')
    # print("Optimal value:",data['optimal_value'],solver.prob.solution.get_objective_value())

# print()
# solver.explain()

In [ ]:
# total_data = []
# from tqdm.auto import tqdm
# for m in tqdm(range(5,31,5)):
# for n,m in [[10,15],[15,20],[20,30],[40,50],[50,100],[100,150],[100,200],[150,200],[200,300]]:
    # if m<50:
    #     pct=0.8
    # else:
    #     pct=1.0
    # data_args = {'n':n,
    #             'm':m,
    #             'low':0,
    #             'high':10,
    #             'bound':1000,
    #             'percent_usage':pct,
    #             'return_solver':True}
    # data, solver = init_data_with_solution(**data_args)
    # # data.update({'solver':solver})
    # total_data.append(data)

### Use CPLEX for optimal solution (given data)

In [9]:
# Example
path_to_read = './drive/MyDrive/Academic/IT4663/6_26_new_data/case_10.txt'
path_to_save = './case_0.txt'
data = read_raw_data_and_solve(path_to_read,path_to_save)
print(data['optimal_path'])
print(data['optimal_value'])
print(data['CPLEX_solving_time'])
# print(data['explanation'])

Warning! There are more than m+n+3 rows in input data
[0, 131, 37, 136, 93, 11, 68, 2, 5, 22, 88, 105, 64, 13, 118, 146, 8, 0]
18
19.944003343582153
